In [3]:
import pandas as pd

df = pd.read_csv("train.csv")
df.describe()

,s6_sensor0_millimeter_step1,s8_sensor0_millimeter_step1,s8_sensor1_millimeter_step1,s7_sensor0_newton_step1,s9_sensor0_kilonewton_step1,s7_sensor1_mikrometer_step1,s0_sensor0_kilonewton_step3,s2_sensor0_millimeter_step1,s9_sensor1_sekunden (zeit)_step1,s8_sensor2_millimeter_step1,...,s8_sensor119_millimeter_step1,s0_sensor57_millimeter_step4,s2_sensor23_millimeter_step1,s2_sensor24_millimeter_step1,s8_sensor120_ml_step1,s7_sensor50_mikrometer_step1,s2_sensor25_millimeter_step1,s8_sensor121_binaer_step1,s8_sensor122_millimeter_step1,s3_sensor3_newtonmeter_step1
count,44022.000000,43531.000000,43531.000000,43389.000000,43498.000000,43733.000000,17253.000000,42866.000000,43396.000000,43554.000000,...,43554.000000,26492.000000,1.000000,42712.000000,44250.000000,43733.000000,42866.000000,43484.000000,43554.000000,43091.000000
mean,5109.904507,20.556290,388.387759,8865.170026,36.836991,31740.544502,-53.537243,18.819547,1056.103706,7.566427,...,166.453309,25843.001210,0.556207,187.826265,160.164095,33783.691037,219.963837,0.070978,757.423916,23.538795
std,628.300759,1.937856,35.189726,12.922544,0.052987,1641.460097,0.000000,1.847855,2.496241,0.918785,...,16.846206,63.369019,NaN,27.564517,9.358430,1202.982083,26.139442,2.181145,58.058404,1.846083
min,3326.894333,0.000000,0.000000,8820.628829,36.836737,20513.790474,-53.537243,-8.785189,1013.999755,0.000000,...,0.000000,24427.685375,0.556207,0.000000,0.000000,29652.075449,-74.220813,0.000000,0.000000,0.000000
25%,4606.469076,19.031959,364.933226,8856.466834,36.836737,31007.643687,-53.537243,18.167772,1056.249745,7.128812,...,161.123531,25845.838372,0.556207,169.949265,156.823842,32962.744067,218.506073,0.000000,718.695812,21.980190
50%,4606.469076,20.440387,387.627804,8865.426335,36.836737,31887.773312,-53.537243,19.195639,1056.249745,7.642628,...,169.068579,25845.838372,0.556207,190.961510,161.135150,33682.454636,222.068672,0.000000,754.378851,23.834538
75%,5886.043819,21.934567,413.162651,8874.385836,36.836737,32767.902936,-53.537243,19.916024,1056.249745,8.125676,...,176.068396,25845.838372,0.556207,207.345508,164.368631,34474.136262,225.557050,0.000000,791.156462,24.508053
max,5886.043819,35.483534,505.347310,8901.264339,47.887758,39808.939931,-53.537243,24.176841,1098.499734,11.568551,...,217.965113,25845.838372,0.556207,292.227576,497.417202,43254.605206,290.203378,67.095788,1092.893408,31.165923


In [4]:
# Step 2: Identify sensor columns
# Assuming sensor column names follow a pattern like 's6_sensor0_millimeter_step1'
sensor_columns = [col for col in df.columns if 'sensor' in col]

# Step 3: Parse sensor columns into components
sensor_metadata = []
for col in sensor_columns:
    parts = col.split('_')  # Split by underscore
    sensor_metadata.append({
        'original_column': col,
        'station': parts[0],   # Extract station (e.g., s6)
        'sensor': parts[1],    # Extract sensor (e.g., sensor0)
        'unit': parts[2],      # Extract unit (e.g., millimeter)
        'step': parts[3]       # Extract step (e.g., step1)
    })

# Convert metadata into a structured DataFrame
sensor_metadata_df = pd.DataFrame(sensor_metadata)

# Step 4: Create a pivot table with stations as columns and sensors as elements
# Adding a binary flag to indicate if the sensor is used
sensor_usage = df[sensor_columns].notnull().astype(int)  # 1 if sensor value exists, 0 if missing
sensor_usage.columns = sensor_columns  # Maintain sensor column names for clarity

# Combine sensor usage data with metadata
sensor_metadata_df['used'] = sensor_usage.sum(axis=0).values  # Sum of "used" for each sensor

# Create a pivot table with stations as columns and sensors as rows
pivot_table = sensor_metadata_df.pivot_table(
    index='sensor',  # Sensors as row headers
    columns='station',  # Stations as column headers
    values='used',  # Value is whether the sensor is used (1) or not (0)
    fill_value=0  # Fill missing values with 0
)

# Step 5: Export the split data into separate tables
# Export the sensor metadata table
sensor_metadata_df.to_csv("sensor_metadata.csv", index=False)

# Export the pivot table with stations and sensors
pivot_table.to_csv("station_sensor_usage.csv")

# Optional: Print tables for verification
print("Sensor Metadata Table:")
print(sensor_metadata_df.head())
print("\nPivot Table (Station vs. Sensors):")
print(pivot_table.head())

Sensor Metadata Table:
               original_column station   sensor        unit   step   used
0  s6_sensor0_millimeter_step1      s6  sensor0  millimeter  step1  44022
1  s8_sensor0_millimeter_step1      s8  sensor0  millimeter  step1  43531
2  s8_sensor1_millimeter_step1      s8  sensor1  millimeter  step1  43531
3      s7_sensor0_newton_step1      s7  sensor0      newton  step1  43389
4  s9_sensor0_kilonewton_step1      s9  sensor0  kilonewton  step1  43498

Pivot Table (Station vs. Sensors):
station         s0       s1      s10       s2       s3       s4       s5  \
sensor                                                                     
sensor0    17253.0      0.0  43704.0  42866.0  43091.0  43291.0  43545.0   
sensor1    17253.0  43451.0  43705.0  42712.0      0.0  43657.0  43545.0   
sensor10   43453.0      0.0      0.0  42866.0      0.0      1.0      0.0   
sensor100      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
sensor101      0.0      0.0      0.0     

In [9]:
# Columns in the dataset
shift_column = "shift"  # Name of the shift column
time_column = "message_timestamp"  # Name of the time column (e.g., timestamp)
sensor_columns = [col for col in df.columns if 'sensor' in col]

# Parse the station names from sensor columns
stations = list(set([col.split('_')[0] for col in sensor_columns]))  # Extract unique station names

# Convert time column to datetime for time-based analysis
df[time_column] = pd.to_datetime(df[time_column])

# Add additional time-based features (e.g., Hour, Day, Week)
df['Hour'] = df[time_column].dt.hour
df['Day'] = df[time_column].dt.day
df['Week'] = df[time_column].dt.isocalendar().week

# Create a table to track station usage by shift and time
station_time_usage = pd.DataFrame(columns=['Shift', 'Station', 'Hour', 'Usage'])

# Iterate through shifts
for shift in df[shift_column].unique():
    # Filter rows for the current shift
    shift_data = df[df[shift_column] == shift]
    
    # For each station, track usage over time
    for station in stations:
        station_columns = [col for col in sensor_columns if col.startswith(station)]
        # Check if the station is used in this shift (non-null sensor values)
        station_usage = shift_data[station_columns].notnull().any(axis=1)
        
        if station_usage.sum() > 0:  # If the station is used at least once
            # Group by Hour and count the number of active sensors
            hourly_usage = shift_data.loc[station_usage, :].groupby('Hour')[station_columns].apply(
                lambda x: x.notnull().sum().sum()
            ).reset_index()
            hourly_usage.columns = ['Hour', 'Usage']
            hourly_usage['Shift'] = shift
            hourly_usage['Station'] = station
            
            # Append the result to the summary DataFrame
            station_time_usage = pd.concat([station_time_usage, hourly_usage], ignore_index=True)

# Export the result to a CSV file for analysis
station_time_usage.to_csv("station_time_usage.csv", index=False)

# Summary table: Station usage by shift and time
print("Station Usage by Shift and Time:")
print(station_time_usage.head())

Station Usage by Shift and Time:
          Shift Station Hour  Usage
0  Nachtschicht      s2    0  38310
1  Nachtschicht      s2    1  24534
2  Nachtschicht      s2    2  40378
3  Nachtschicht      s2    3  33072
4  Nachtschicht      s2    4  27120


In [11]:
# 1. Change "Physical_part_type" Column
# Map original types (e.g., Type 1, 2, 4) to numerical values (1-3)
part_type_mapping = {"type1": 1, "type2": 2, "type4": 3}
df['physical_part_type'] = df['physical_part_type'].map(part_type_mapping)

# 2. Change "message_timestamp" Column
# Convert timestamp to datetime format
df['message_timestamp'] = pd.to_datetime(df['message_timestamp'])

# Extract Month as 1-3 for September, October, November
month_mapping = {9: 1, 10: 2, 11: 3}
df['Month'] = df['message_timestamp'].dt.month.map(month_mapping)

# Divide Month into Weeks (1-4 based on day ranges)
df['Week'] = ((df['message_timestamp'].dt.day - 1) // 7 + 1)

# Convert Daytime to Seconds Since Midnight
df['Daytime'] = df['message_timestamp'].dt.hour * 3600 + \
                df['message_timestamp'].dt.minute * 60 + \
                df['message_timestamp'].dt.second

# 3. Change "Weekday" Column
# Map weekday names to numerical values (Monday=1, ..., Sunday=6)
weekday_mapping = {
    'Sunday': 1, 'Monday': 2, 'Tuesday': 3,
    'Wednesday': 4, 'Thursday': 5, 'Friday': 6
}
df['weekday'] = df['message_timestamp'].dt.day_name().map(weekday_mapping)

# 4. Change "shift" Column
# Map shift types to numerical values (Early=1, Late=2, Night=3)
shift_mapping = {"Fruehschicht": 1, "Spaetschicht": 2, "Nachtschicht": 3}
df['shift'] = df['shift'].map(shift_mapping)

# 5. Change "Status" Column
# Map status to binary values (e.g., OK=1, NOK=0)
status_mapping = {"OK": 1, "NOK": 0}
df['status'] = df['status'].map(status_mapping)

# Drop original timestamp column after transformations (optional)
df = df.drop(columns=['message_timestamp'])

# Save the transformed dataset to a new file
df.to_csv("transformed_data.csv", index=False)

# Display the first few rows of the transformed dataset
print("Transformed Dataset:")
print(df.head())

Transformed Dataset:
   physical_part_type  weekday  shift  status  \
0                 1.0        1      3       1   
1                 2.0        1      3       1   
2                 1.0        1      3       1   
3                 1.0        1      3       1   
4                 1.0        1      3       1   

                                    physical_part_id  \
0  gAAAAABnRuZgg4Kol8URwnouW9Hv5RymnFqydE_5oyUU_3...   
1  gAAAAABnRuZfvjtCL7qj_QEGC47fSq-n7brsZXoEV--vJ1...   
2  gAAAAABnRuZfuXBrn_3FsgN6QXP6DQZPeVgP17o6yPbv4h...   
3  gAAAAABnRuZheONxFxB3r6dyfTZt1ehdUXm6a-9Opa35i4...   
4  gAAAAABnRuZiqnRUJiO7HqQmkkVmg-m21um0EcSExrMdud...   

   s6_sensor0_millimeter_step1  s8_sensor0_millimeter_step1  \
0                  4606.469076                    19.525688   
1                  5886.043819                    17.540378   
2                  4606.469076                    20.445584   
3                  4606.469076                    20.822377   
4                  4606.469076  

In [13]:
# Check if Physical_part_id is unique
if 'physical_part_id' in df.columns:
    # Count the number of unique Physical Part IDs
    total_ids = len(df['physical_part_id'])
    unique_ids = df['physical_part_id'].nunique()
    duplicate_ids = total_ids - unique_ids

    print(f"Total Physical Part IDs: {total_ids}")
    print(f"Unique Physical Part IDs: {unique_ids}")
    print(f"Duplicate Physical Part IDs: {duplicate_ids}")

    # Identify and display repeated Physical Part IDs
    if duplicate_ids > 0:
        repeated_ids = df['physical_part_id'][df['physical_part_id'].duplicated()]
        print("\nRepeated Physical Part IDs:")
        print(repeated_ids.value_counts())
    else:
        print("\nAll Physical Part IDs are unique.")
else:
    print("The column 'Physical_part_id' does not exist in the dataset.")

Total Physical Part IDs: 44818
Unique Physical Part IDs: 44818
Duplicate Physical Part IDs: 0

All Physical Part IDs are unique.
